In [2]:
#Surprise
from surprise import SVD, KNNBasic, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise.model_selection import KFold
from surprise import accuracy
from surprise.accuracy import rmse
# Importar librerías necesarias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import copy
from collections import defaultdict
# Librerías de Scikit-learn
from sklearn import datasets
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from surprise.accuracy import rmse

In [5]:
df = pd.read_csv("C:\\Users\\guill\\Downloads\\ratings_Electronics.csv", names=['userId', 'productId','rating','timestamp'])

In [6]:
#Filtratge del dataset
# Comptem quantes valoracions ha fet cada usuari i quantes valoracions ha rebut cada producte
user_counts = df['userId'].value_counts()  # Nombre total de valoracions per a cada usuari
product_counts = df['productId'].value_counts()  # Nombre total de valoracions per a cada producte

# Filtrar el dataset:
# - Mantenim només els usuaris que han fet almenys 50 valoracions
# - Mantenim només els productes que han rebut almenys 5 valoracions
filtered_dataset = df[(df['userId'].isin(user_counts[user_counts >= 50].index)) &
                      (df['productId'].isin(product_counts[product_counts >= 5].index))]

# Imprimir estadístiques del dataset filtrat
print("Nombre total de valoracions:", filtered_dataset['rating'].nunique())  # Total de valoracions diferents
print("Nombre total d'usuaris únics:", filtered_dataset['userId'].nunique())  # Nombre d'usuaris únics
print("Nombre total de productes únics:", filtered_dataset['productId'].nunique())  # Nombre de productes únics
print("Nombre total d'usuaris:", filtered_dataset['userId'])  # Nombre d'usuaris
print("Nombre total de productes:", filtered_dataset['productId'])  # Nombre de productes



Nombre total de valoracions: 5
Nombre total d'usuaris únics: 1540
Nombre total de productes únics: 39939
Nombre total d'usuaris: 94         A3BY5KCNQZXV5U
118         AT09WGFUM934H
177        A32HSNCNPRUMTR
178        A17HMM1M7T9PJ1
492        A3CLWR1UUZT6TG
                ...      
7824423    A34BZM6S9L7QI4
7824424    A1G650TTTHEAL5
7824425    A25C2M3QF9G7OQ
7824426     A1E1LEVQ9VQNK
7824427    A2NYK9KWFMJV4Y
Name: userId, Length: 117315, dtype: object
Nombre total de productes: 94         0594451647
118        0594481813
177        0970407998
178        0970407998
492        0972683275
              ...    
7824423    B00LGQ6HL8
7824424    B00LGQ6HL8
7824425    B00LGQ6HL8
7824426    B00LGQ6HL8
7824427    B00LGQ6HL8
Name: productId, Length: 117315, dtype: object


In [ ]:
# Definir el rang de les qualificacions del dataset
reader = Reader(rating_scale=(1, 5))

# Convertir el dataset filtrat al format compatible amb la llibreria Surprise
data = Dataset.load_from_df(filtered_dataset[['userId', 'productId', 'rating']], reader)

"""# Crear el conjunt complet d'entrenament a partir del dataset
full_trainset = data.build_full_trainset()"""

# Dividir el dataset en conjunts d'entrenament i prova (80% entrenament, 20% prova)
trainset, testset = train_test_split(data, test_size=0.2)

svd = SVD()
svd.fit(trainset)
pred = svd.test(testset)

#User-User
sim_options_cosinus = {'name': 'cosine', 'user_based': True} #Distancia cosinus
sim_options_pearson = {'name': 'pearson', 'user_based': True} #Distancia pearson

#Item-Item
sim_options_cosinus_items = {'name': 'cosine', 'user_based': False} #Distancia cosinus
sim_options_pearson_items = {'name': 'pearson', 'user_based': False} #Distancia pearson

In [ ]:
# Models
model_cosinus_i = KNNBasic(k=5, sim_options=sim_options_cosinus_items)  # Model Item-Item amb similitud cosinus
model_cosinus_u = KNNBasic(k=5, sim_options=sim_options_cosinus)  # Model User-User amb similitud cosinus
model_pearson_u = KNNBasic(k=5, sim_options=sim_options_pearson)  # Model User-User amb similitud Pearson
model_pearson_i = KNNBasic(k=5, sim_options=sim_options_pearson_items)  # Model Item-Item amb similitud Pearson
svd = SVD(n_factors = 5)  # Model basat en descomposició de valors singulars (SVD)

# Entrenar els models amb el conjunt d'entrenament i fer prediccions pel conjunt de test. Mesurem el temps d'execució dels models.

start_time = time.time()
model_cosinus_i.fit(trainset)
predictions_cosinus_i = model_cosinus_i.test(testset)
time_cosinus_i = time.time() - start_time
start_time = time.time()
model_cosinus_u.fit(trainset)
predictions_cosinus_u = model_cosinus_u.test(testset)
time_cosinus_u = time.time() - start_time
start_time = time.time()
model_pearson_u.fit(trainset)
predictions_pearson_u = model_pearson_u.test(testset)
time_pearson_u = time.time() - start_time
start_time = time.time()
model_pearson_i.fit(trainset)
predictions_pearson_i = model_pearson_i.test(testset)
time_pearson_i = time.time() - start_time
start_time = time.time()
svd.fit(trainset)
pred = svd.test(testset)
time_svd = time.time() - start_time



In [ ]:
# Calcular RMSE i MAE per a cada model
# Es mesuren les mètriques d'error (RMSE i MAE) per a cadascun dels models amb les prediccions realitzades.

print("Model Cosinus Items:")
rmse_c_i = accuracy.rmse(predictions_cosinus_i, verbose=True)  # RMSE per al model basat en distància cosinus (Item-Item)
mae_c_i = accuracy.mae(predictions_cosinus_i, verbose=True)  # MAE per al model basat en distància cosinus (Item-Item)

print("\nModel Cosinus Users:")
rmse_c_u = accuracy.rmse(predictions_cosinus_u, verbose=True)  # RMSE per al model basat en distància cosinus (User-User)
mae_c_u = accuracy.mae(predictions_cosinus_u, verbose=True)  # MAE per al model basat en distància cosinus (User-User)

print("\nModel Pearson Users:")
rmse_pearson_u = accuracy.rmse(predictions_pearson_u, verbose=True)  # RMSE per al model basat en correlació Pearson (User-User)
mae_pearson_u = accuracy.mae(predictions_pearson_u, verbose=True)  # MAE per al model basat en correlació Pearson (User-User)

print("\nModel Pearson Items:")
rmse_pearson_i = accuracy.rmse(predictions_pearson_i, verbose=True)  # RMSE per al model basat en correlació Pearson (Item-Item)
mae_pearson_i = accuracy.mae(predictions_pearson_i, verbose=True)  # MAE per al model basat en correlació Pearson (Item-Item)

print("\nModel SVD:")
rmse_svd = accuracy.rmse(pred, verbose=True)  # RMSE per al model SVD
mae_svd = accuracy.mae(pred, verbose=True)  # MAE per al model SVD

# Mostrar els resultats
# Es presenten els resultats finals de les mètriques calculades (RMSE i MAE) per a cada model.
print("\nResum de RMSE i MAE:")
print(f"Cosinus Items - RMSE: {rmse_c_i:.4f}, MAE: {mae_c_i:.4f}")
print(f"Cosinus Users - RMSE: {rmse_c_u:.4f}, MAE: {mae_c_u:.4f}")
print(f"Pearson Users - RMSE: {rmse_pearson_u:.4f}, MAE: {mae_pearson_u:.4f}")
print(f"Pearson Items - RMSE: {rmse_pearson_i:.4f}, MAE: {mae_pearson_i:.4f}")
print(f"SVD - RMSE: {rmse_svd:.4f}, MAE: {mae_svd:.4f}")


In [ ]:

unics = filtered_dataset['productId'].unique()

np.random.seed(42) #Al estar agafant usuaris aleatòris, cal assegurar la reproductibilitat de les dades.

un_terç = len(unics) // 3
dos_terços = len(unics) // 3 * 2

items_1_terç = np.random.choice(unics, size=un_terç, replace=False) #Seleccionem un terç dels items

dataset_1_terç = filtered_dataset[filtered_dataset['productId'].isin(items_1_terç)]

items_2_terços = np.random.choice(unics, size=dos_terços, replace=False) #Seleccionem dos terços dels items

dataset_2_terços = filtered_dataset[filtered_dataset['productId'].isin(items_2_terços)]


In [ ]:

#Divisió dels datasets

data_1_terç = Dataset.load_from_df(dataset_1_terç[['userId','productId', 'rating']], reader)

data_2_terços = Dataset.load_from_df(dataset_2_terços[['userId','productId', 'rating']], reader)

train_1,test_1 = train_test_split(data_1_terç, test_size=0.2)

train_2,test_2 = train_test_split(data_2_terços, test_size=0.2)

#Entrenament dels models

model_cosinus_u.fit(train_1)
model_cosinus_i.fit(train_1)
model_cosinus_u.fit(train_2)
model_cosinus_i.fit(train_2)
svd.fit(train_1)
svd.fit(train_2)

#Càlcul de prediccions

predictions_cosinus_i_500 = model_cosinus_i.test(test_1)
predictions_cosinus_u_500 = model_cosinus_u.test(test_1)
predictions_cosinus_i_1000 = model_cosinus_i.test(test_2)
predictions_cosinus_u_1000 = model_cosinus_u.test(test_2)
pred_500 = svd.test(test_1)
pred_1000 = svd.test(test_2)

#Calcular els errors (RMSE)

rmse_c_i_500 = accuracy.rmse(predictions_cosinus_i_500)
rmse_c_u_500 = accuracy.rmse(predictions_cosinus_u_500)
rmse_c_i_1000 = accuracy.rmse(predictions_cosinus_i_1000)
rmse_c_u_1000 = accuracy.rmse(predictions_cosinus_u_1000)
rmse_svd_500 = accuracy.rmse(pred_500)
rmse_svd_1000 = accuracy.rmse(pred_1000)

models = ['Dataset 1 terç', 'Dataset 2 terços', 'Dataset Filtrat']
valors_rmse_u = [rmse_c_u_500, rmse_c_u_1000, rmse_c_u]
valors_rmse_i = [rmse_c_i_500,rmse_c_i_1000, rmse_c_i]
valors_rmse_svd = [rmse_svd_500,rmse_svd_1000, rmse_svd]

# Graficar comparació de RMSE

#USER USER
plt.figure(figsize=(8, 6))
plt.bar(models, valors_rmse_u, color=['blue', 'green', 'red'])
plt.title('Comparació de RMSE entre models amb diferents datasets', fontsize=14)
plt.ylabel('RMSE', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#ITEM ITEM
plt.figure(figsize=(8, 6))
plt.bar(models, valors_rmse_i, color=['blue', 'green', 'red'])
plt.title('Comparació de RMSE entre models amb diferents datasets', fontsize=14)
plt.ylabel('RMSE', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#SVD
plt.figure(figsize=(8, 6))
plt.bar(models, valors_rmse_svd, color=['blue', 'green', 'red'])
plt.title('Comparació de RMSE entre models amb diferents datasets', fontsize=14)
plt.ylabel('RMSE', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()